# Group Name: Bill Greats 

#### Team Members: Kumar Kishalaya, Sakshi Arya, Shiv Goel, Ruben Casas


In [2]:
pip install surprise

  Obtaining dependency information for surprise from https://files.pythonhosted.org/packages/61/de/e5cba8682201fcf9c3719a6fdda95693468ed061945493dea2dd37c5618b/surprise-0.1-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-macosx_11_0_arm64.whl size=485168 sha256=4b46f977121d2ec88fa200fcffde2fa74d3770f2e934291f92f43141fa9b3495
  Stored in directory: /Users/kumarkishalaya/Library/Caches/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import train_test_split
from collections import defaultdict

# Load the data
df_final = pd.read_csv('df_ratings.csv')

# Ensure the data is in the right format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_final[['user_id', 'movie_id', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Use user-based collaborative filtering
algo = KNNBasic(sim_options={'user_based': True})

# Train the algorithm on the trainset
algo.fit(trainset)

# Predict the ratings for the testset
predictions = algo.test(testset)

# Function to get the top-N recommendations for each user
def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Get the top-5 recommendations for each user
top_n = get_top_n(predictions, n=5)

# Function to calculate the precision@k
def precision_at_k(predictions, k=5):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= 3.5) for (_, true_r) in user_ratings)
        n_rec_k = sum((true_r >= 3.5) for (_, true_r) in user_ratings[:k])
        precisions[uid] = n_rec_k / k if n_rel != 0 else 0

    return sum(prec for prec in precisions.values()) / len(precisions)

# Calculate top-5 precision
top_5_precision = precision_at_k(predictions, k=5)
print(f'Top-5 Precision: {top_5_precision:.4f}')


Computing the msd similarity matrix...
Done computing similarity matrix.
Top-5 Precision: 0.9169
